In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re

## Step 1: Load Players and Get Ranking IDs

In [ ]:
print("Loading player data...")

# Load players with ELO_KLASSIERUNG between 11 and 21
female_df = pd.read_csv('../data/raw/elo-rankings_female_20251218.csv', sep=';', encoding='latin-1')
female_df = female_df[(female_df['ELO_KLASSIERUNG'] >= 11) & (female_df['ELO_KLASSIERUNG'] <= 21)]

male_df = pd.read_csv('../data/raw/elo-rankings_male_20251218.csv', sep=';', encoding='latin-1')
male_df = male_df[(male_df['ELO_KLASSIERUNG'] >= 11) & (male_df['ELO_KLASSIERUNG'] <= 21)]

all_players = pd.concat([female_df, male_df], ignore_index=True)
print(f"Found {len(all_players)} players with ELO_KLASSIERUNG 11-21")

In [ ]:
print("Collecting ranking IDs...")
search_url = "https://www.click-tt.ch/cgi-bin/WebObjects/nuLigaTTCH.woa/wa/eloFilter"
ranking_ids = []

for i, player in all_players.iterrows():
    if (i + 1) % 100 == 0:
        print(f"  {i + 1}/{len(all_players)}...")
    
    try:
        response = requests.get(search_url, params={
            "federation": "STT",
            "rankingDate": "18.12.2025",
            "lastname": player['NACHNAME'],
            "firstname": player['VORNAME']
        }, timeout=10)
        soup = BeautifulSoup(response.text, 'lxml')
        
        for link in soup.find_all('a'):
            href = link.get('href', '')
            match = re.search(r'ranking=(\d+)', href)
            if match:
                ranking_ids.append(match.group(1))
                break
    except:
        pass
    
    time.sleep(0.2)

ranking_ids = list(set(ranking_ids))
print(f"Found {len(ranking_ids)} unique ranking IDs")

## Step 2: Fetch Matches for Each Ranking ID

In [ ]:
print("Scraping matches...")

base_url = "https://www.click-tt.ch/cgi-bin/WebObjects/nuLigaTTCH.woa/wa/eloFilter?federation=STT&rankingDate=18.12.2025&ranking="

all_matches = []

for i, ranking_id in enumerate(ranking_ids):
    
    if (i + 1) % 50 == 0:
        print(f"  {i + 1}/{len(ranking_ids)} players... ({len(all_matches)} matches)")
    
    url = base_url + ranking_id
    
    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.text, 'lxml')
    except:
        continue
    
    for table in soup.find_all('table'):
        for row in table.find_all('tr'):
            cells = row.find_all('td')
            if len(cells) < 7:
                continue
            
            texts = [c.get_text(strip=True) for c in cells]
            
            if not re.match(r'\d{2}\.\d{2}\.\d{4}', texts[0]):
                continue
            
            try:
                player_elo = float(texts[2].replace(',', '.'))
                opponent_elo = float(texts[4].replace(',', '.'))
                expected_prob = float(texts[5].replace(',', '.'))
                elo_delta = float(texts[6].replace(',', '.'))
                win = 1 if elo_delta > 0 else 0
                
                all_matches.append({
                    'player_elo': player_elo,
                    'opponent_elo': opponent_elo,
                    'win': win,
                    'expected_prob': expected_prob,
                    'elo_delta': elo_delta
                })
            except:
                continue
    
    time.sleep(0.2)

## Step 3: Save to CSV

In [ ]:
print(f"Total matches: {len(all_matches)}")

df = pd.DataFrame(all_matches)
df = df.drop_duplicates()
df.to_csv('../data/raw/historical_matches.csv', index=False)

print(f"Saved {len(df)} unique matches to data/raw/historical_matches.csv")

if len(df) > 0:
    print(f"\nSummary:")
    print(f"  Player Elo: {df['player_elo'].min():.0f} - {df['player_elo'].max():.0f}")
    print(f"  Opponent Elo: {df['opponent_elo'].min():.0f} - {df['opponent_elo'].max():.0f}")
    print(f"  Win rate: {df['win'].mean()*100:.1f}%")

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re

## Step 1: Load Players and Get Ranking IDs

In [ ]:
print("Loading player data...")

# Load players with ELO_KLASSIERUNG between 11 and 21
female_df = pd.read_csv('../data/raw/elo-rankings_female_20251218.csv', sep=';', encoding='latin-1')
female_df = female_df[(female_df['ELO_KLASSIERUNG'] >= 11) & (female_df['ELO_KLASSIERUNG'] <= 21)]

male_df = pd.read_csv('../data/raw/elo-rankings_male_20251218.csv', sep=';', encoding='latin-1')
male_df = male_df[(male_df['ELO_KLASSIERUNG'] >= 11) & (male_df['ELO_KLASSIERUNG'] <= 21)]

all_players = pd.concat([female_df, male_df], ignore_index=True)
print(f"Found {len(all_players)} players with ELO_KLASSIERUNG 11-21")

# Show sample
all_players[['VORNAME', 'NACHNAME', 'ELO_WERT', 'ELO_KLASSIERUNG']].head(10)

In [ ]:
# Get ranking IDs by searching each player's name
print("Collecting ranking IDs...")
search_url = "https://www.click-tt.ch/cgi-bin/WebObjects/nuLigaTTCH.woa/wa/eloFilter"
ranking_ids = []

# NOTE: For demo, only process first 10 players (full scrape takes ~20 min)
demo_players = all_players.head(10)

for i, player in demo_players.iterrows():
    print(f"  Searching: {player['VORNAME']} {player['NACHNAME']}...")
    
    try:
        response = requests.get(search_url, params={
            "federation": "STT",
            "rankingDate": "18.12.2025",
            "lastname": player['NACHNAME'],
            "firstname": player['VORNAME']
        }, timeout=10)
        soup = BeautifulSoup(response.text, 'lxml')
        
        for link in soup.find_all('a'):
            href = link.get('href', '')
            match = re.search(r'ranking=(\d+)', href)
            if match:
                ranking_ids.append(match.group(1))
                print(f"    Found ranking ID: {match.group(1)}")
                break
    except Exception as e:
        print(f"    Error: {e}")
    
    time.sleep(0.2)  # Be nice to the server

ranking_ids = list(set(ranking_ids))  # Remove duplicates
print(f"\nFound {len(ranking_ids)} unique ranking IDs")

## Step 2: Fetch Matches for Each Ranking ID

In [ ]:
print("Scraping matches...")

# Base URL - only the ranking ID changes
base_url = "https://www.click-tt.ch/cgi-bin/WebObjects/nuLigaTTCH.woa/wa/eloFilter?federation=STT&rankingDate=18.12.2025&ranking="

all_matches = []

for i, ranking_id in enumerate(ranking_ids):
    print(f"  Player {i + 1}/{len(ranking_ids)} (ranking ID: {ranking_id})...")
    
    # Simple URL: base + ranking_id
    url = base_url + ranking_id
    
    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.text, 'lxml')
    except:
        continue
    
    # Find match rows in tables
    matches_found = 0
    for table in soup.find_all('table'):
        for row in table.find_all('tr'):
            cells = row.find_all('td')
            if len(cells) < 7:
                continue
            
            texts = [c.get_text(strip=True) for c in cells]
            
            # First cell must be a date (DD.MM.YYYY format)
            if not re.match(r'\d{2}\.\d{2}\.\d{4}', texts[0]):
                continue
            
            try:
                player_elo = float(texts[2].replace(',', '.'))
                opponent_elo = float(texts[4].replace(',', '.'))
                expected_prob = float(texts[5].replace(',', '.'))
                elo_delta = float(texts[6].replace(',', '.'))
                win = 1 if elo_delta > 0 else 0
                
                all_matches.append({
                    'player_elo': player_elo,
                    'opponent_elo': opponent_elo,
                    'win': win,
                    'expected_prob': expected_prob,
                    'elo_delta': elo_delta
                })
                matches_found += 1
            except:
                continue
    
    print(f"    Found {matches_found} matches")
    time.sleep(0.2)  # Be nice to the server

print(f"\nTotal matches scraped: {len(all_matches)}")

## Step 3: Save to CSV

In [ ]:
# Convert to DataFrame and remove duplicates
df = pd.DataFrame(all_matches)
df = df.drop_duplicates()

print(f"Total matches: {len(all_matches)}")
print(f"Unique matches: {len(df)}")

if len(df) > 0:
    print(f"\nSummary:")
    print(f"  Player Elo: {df['player_elo'].min():.0f} - {df['player_elo'].max():.0f}")
    print(f"  Opponent Elo: {df['opponent_elo'].min():.0f} - {df['opponent_elo'].max():.0f}")
    print(f"  Win rate: {df['win'].mean()*100:.1f}%")

df.head(20)

In [ ]:
# Uncomment to save (don't overwrite the full dataset with demo data!)
# df.to_csv('../data/raw/historical_matches.csv', index=False)
# print("Saved to data/raw/historical_matches.csv")

## Compare with Full Dataset

The full scrape (all 623 players) produced 31,584 matches:

In [ ]:
# Load the complete scraped dataset
full_df = pd.read_csv('../data/raw/historical_matches.csv')

print(f"Full dataset: {len(full_df)} matches")
print(f"\nSummary:")
print(f"  Player Elo: {full_df['player_elo'].min():.0f} - {full_df['player_elo'].max():.0f}")
print(f"  Opponent Elo: {full_df['opponent_elo'].min():.0f} - {full_df['opponent_elo'].max():.0f}")
print(f"  Win rate: {full_df['win'].mean()*100:.1f}%")

full_df.head(10)

## Step 1: Understanding the Data Source

The Swiss Table Tennis website has an **Elo ranking page** for each player.

Each player has a unique `ranking` ID. For example:
- Player with ranking ID `12345` has URL: `https://www.click-tt.ch/.../eloFilter?...&ranking=12345`

On this page, we can see:
- The player's Elo rating
- Their match history with other players
- Win/loss results and Elo changes

## Step 2: Load Players from CSV

We have CSV files with all Swiss players and their Elo ratings.
We filter for players with `ELO_KLASSIERUNG` between 11 and 21 (similar level to our focal player).

In [2]:
# Load female and male rankings
female_df = pd.read_csv('../data/raw/elo-rankings_female_20251218.csv', sep=';', encoding='latin-1')
male_df = pd.read_csv('../data/raw/elo-rankings_male_20251218.csv', sep=';', encoding='latin-1')

print(f"Total female players: {len(female_df)}")
print(f"Total male players: {len(male_df)}")

# Show columns
print(f"\nColumns: {list(female_df.columns)}")

Total female players: 432
Total male players: 4921

Columns: ['NACHNAME', 'VORNAME', 'LIZENZNR', 'VEREIN', 'ELO_WERT', 'ELO_WERT_DELTA', 'PLATZIERUNG', 'ELO_KLASSIERUNG', 'ELO_KLASSIERUNG_HERREN']


In [3]:
# Filter for ELO_KLASSIERUNG 11-21
female_filtered = female_df[(female_df['ELO_KLASSIERUNG'] >= 11) & (female_df['ELO_KLASSIERUNG'] <= 21)]
male_filtered = male_df[(male_df['ELO_KLASSIERUNG'] >= 11) & (male_df['ELO_KLASSIERUNG'] <= 21)]

all_players = pd.concat([female_filtered, male_filtered], ignore_index=True)
print(f"Players with ELO_KLASSIERUNG 11-21: {len(all_players)}")

# Show sample
all_players[['VORNAME', 'NACHNAME', 'ELO_WERT', 'ELO_KLASSIERUNG']].head(10)

Players with ELO_KLASSIERUNG 11-21: 623


,VORNAME,NACHNAME,ELO_WERT,ELO_KLASSIERUNG
0,Lucie,Gauthier,1738,21
1,Rachel,Moret,1735,21
2,Bérénice,Marteau,1573,20
3,Monika,Pietkiewicz,1566,20
4,Olga,Nemes,1563,20
5,Marion,Berthaud,1538,20
6,Fanny,Doutaz,1524,20
7,Alex,Blazek,1509,20
8,Laura,Robertson,1498,20
9,Ruoqi,Wei,1416,19


## Step 3: How to Find a Player's Ranking ID

The problem: Our CSV has player names, but the website uses `ranking` IDs.

Solution: **Search for the player by name** → the result page contains a link with their ranking ID.

Let's try with one player:

In [8]:
# Example: Search for a specific player
search_url = "https://www.click-tt.ch/cgi-bin/WebObjects/nuLigaTTCH.woa/wa/eloFilter"

# Pick a player from our list
example_player = all_players.iloc[2]
print(f"Searching for: {example_player['VORNAME']} {example_player['NACHNAME']}")
print(f"Expected Elo: {example_player['ELO_WERT']}")

Searching for: Bérénice Marteau
Expected Elo: 1573


In [21]:
# Send search request
response = requests.get(search_url, params={
    "federation": "STT",
    "rankingDate": "18.12.2025",
    "lastname": example_player['NACHNAME'],
    "firstname": example_player['VORNAME']
}, timeout=10)

print(f"Status code: {response.status_code}")
print(f"Response length: {len(response.text)} characters")

Status code: 200
Response length: 11541 characters


In [22]:
# Parse HTML with BeautifulSoup
soup = BeautifulSoup(response.text, 'lxml')

# Find all links and look for ranking=XXXXX pattern
ranking_id = None
for link in soup.find_all('a'):
    href = link.get('href', '')
    match = re.search(r'ranking=(\d+)', href)
    if match:
        ranking_id = match.group(1)
        print(f"Found ranking ID: {ranking_id}")
        print(f"Full URL: {href[:100]}...")
        break

if not ranking_id:
    print("No ranking ID found!")

Found ranking ID: 368580165
Full URL: /cgi-bin/WebObjects/nuLigaTTCH.woa/wa/eloFilter?lastname=Marteau&firstname=B%C3%A9r%C3%A9nice&federa...


## Step 4: Fetch Match History for a Player

Now that we have the ranking ID, we can fetch the player's match history.

The page contains a table with all their matches.

In [23]:
if ranking_id:
    # Fetch player's page
    player_url = f"https://www.click-tt.ch/cgi-bin/WebObjects/nuLigaTTCH.woa/wa/eloFilter?federation=STT&rankingDate=18.12.2025&ranking={ranking_id}"
    
    response = requests.get(player_url, timeout=10)
    soup = BeautifulSoup(response.text, 'lxml')
    
    print(f"Fetched page for ranking ID: {ranking_id}")
    print(f"Page length: {len(response.text)} characters")

Fetched page for ranking ID: 368580165
Page length: 43962 characters


In [24]:
# Find all tables on the page
tables = soup.find_all('table')
print(f"Found {len(tables)} tables on the page")

Found 21 tables on the page


In [25]:
# Extract matches from tables
matches = []

for table in tables:
    for row in table.find_all('tr'):
        cells = row.find_all('td')
        if len(cells) < 7:
            continue
        
        # Get text from each cell
        texts = [c.get_text(strip=True) for c in cells]
        
        # First cell must be a date (DD.MM.YYYY format)
        if not re.match(r'\d{2}\.\d{2}\.\d{4}', texts[0]):
            continue
        
        # This looks like a match row!
        print(f"Match row: {texts[:7]}")
        
        try:
            # Parse the values
            # texts[0] = date
            # texts[1] = opponent name
            # texts[2] = player Elo
            # texts[3] = opponent name again or club
            # texts[4] = opponent Elo
            # texts[5] = expected probability
            # texts[6] = Elo delta
            
            player_elo = float(texts[2].replace(',', '.'))
            opponent_elo = float(texts[4].replace(',', '.'))
            expected_prob = float(texts[5].replace(',', '.'))
            elo_delta = float(texts[6].replace(',', '.'))
            win = 1 if elo_delta > 0 else 0
            
            matches.append({
                'date': texts[0],
                'player_elo': player_elo,
                'opponent_elo': opponent_elo,
                'expected_prob': expected_prob,
                'elo_delta': elo_delta,
                'win': win
            })
        except:
            pass

print(f"\nExtracted {len(matches)} matches")

Match row: ['13.12.2025', 'STTL-Woman | Rio-Star Muttenz : ZZ-Lancy', '1584', 'Kühn, Emma (C10 / B15)', '1153', '0,993', '0,104']
Match row: ['13.12.2025', 'STTL-Woman | Rio-Star Muttenz : ZZ-Lancy', '1584', 'Hu, Elina Jiale (C8 / B13)', '1044', '0,998', '0,030']
Match row: ['13.12.2025', 'STTL-Woman | Rio-Star Muttenz : ZZ-Lancy', '1584', 'Blazek, Alex (A17 / A20)', '1497', '0,731', '-10,971']
Match row: ['09.11.2025', 'STTL-Woman | ZZ-Lancy : Zürich-Affoltern', '1608', 'Geiger, Margit (C8 / B13)', '1062', '0,998', '0,028']
Match row: ['09.11.2025', 'STTL-Woman | ZZ-Lancy : Zürich-Affoltern', '1608', 'Wei, Ruoqi (B15 / A19)', '1347', '0,953', '-14,292']
Match row: ['09.11.2025', 'STTL-Woman | ZZ-Lancy : Zürich-Affoltern', '1608', 'Nemes, Olga (A17 / A20)', '1542', '0,681', '-10,220']
Match row: ['08.11.2025', 'STTL-Woman | ZZ-Lancy : Young Stars ZH', '1608', 'Wildberger, Jill (C8 / B13)', '1075', '0,998', '0,032']
Match row: ['08.11.2025', 'STTL-Woman | ZZ-Lancy : Young Stars ZH', '16

In [26]:
# Show extracted matches
if matches:
    match_df = pd.DataFrame(matches)
    display(match_df.head(10))

,date,player_elo,opponent_elo,expected_prob,elo_delta,win
0,13.12.2025,1584.0,1153.0,0.993,0.104,1
1,13.12.2025,1584.0,1044.0,0.998,0.030,1
2,13.12.2025,1584.0,1497.0,0.731,-10.971,0
3,09.11.2025,1608.0,1062.0,0.998,0.028,1
4,09.11.2025,1608.0,1347.0,0.953,-14.292,0
5,09.11.2025,1608.0,1542.0,0.681,-10.220,0
6,08.11.2025,1608.0,1075.0,0.998,0.032,1
7,08.11.2025,1608.0,1083.0,0.998,0.035,1
8,08.11.2025,1608.0,1327.0,0.962,0.568,1
9,24.05.2025,1612.0,1341.0,0.958,0.634,1


## Step 5: Understanding the Match Data

Each match row contains:

| Column | Description |
|--------|-------------|
| `date` | When the match was played |
| `player_elo` | Player's Elo at time of match |
| `opponent_elo` | Opponent's Elo at time of match |
| `expected_prob` | Expected win probability (Swiss formula) |
| `elo_delta` | Change in Elo after match (+ = win, - = loss) |
| `win` | 1 if won, 0 if lost |

In [27]:
# Verify the Swiss Elo formula
if matches:
    sample = matches[0]
    
    # Swiss Elo probability formula: P = 1 / (1 + 10^((Ej - Ei) / 200))
    player_elo = sample['player_elo']
    opponent_elo = sample['opponent_elo']
    
    calculated_prob = 1 / (1 + 10 ** ((opponent_elo - player_elo) / 200))
    
    print(f"Player Elo: {player_elo}")
    print(f"Opponent Elo: {opponent_elo}")
    print(f"Expected prob (from website): {sample['expected_prob']}")
    print(f"Calculated prob (our formula): {calculated_prob:.4f}")
    print(f"\nMatch? {abs(calculated_prob - sample['expected_prob']) < 0.01}")

Player Elo: 1584.0
Opponent Elo: 1153.0
Expected prob (from website): 0.993
Calculated prob (our formula): 0.9931

Match? True


## Step 6: The Full Scraping Process

The full scraper (`scrape_matches.py`) does this for ALL players:

1. **Load players** from CSV files (filter ELO_KLASSIERUNG 11-21)
2. **Search each player** to get their ranking ID
3. **Fetch each player's page** and extract matches from tables
4. **Save all matches** to CSV (removing duplicates)

Result: **31,584 matches** for training the ML model!

In [28]:
# Load the scraped data
historical = pd.read_csv('../data/raw/historical_matches.csv')

print(f"Total matches scraped: {len(historical)}")
print(f"\nSummary:")
print(f"  Player Elo range: {historical['player_elo'].min():.0f} - {historical['player_elo'].max():.0f}")
print(f"  Opponent Elo range: {historical['opponent_elo'].min():.0f} - {historical['opponent_elo'].max():.0f}")
print(f"  Win rate: {historical['win'].mean()*100:.1f}%")

historical.head(10)

Total matches scraped: 31584

Summary:
  Player Elo range: 793 - 1958
  Opponent Elo range: 492 - 1958
  Win rate: 63.0%


,player_elo,opponent_elo,win,expected_prob,elo_delta
0,1736.0,1310.0,1,0.993,0.110
1,1736.0,1346.0,1,0.989,0.166
2,1736.0,1327.0,1,0.991,0.134
3,1736.0,1417.0,1,0.975,0.372
4,1736.0,1368.0,1,0.986,0.214
5,1736.0,1384.0,1,0.983,0.256
6,1736.0,1107.0,1,0.999,0.011
7,1736.0,1508.0,1,0.932,1.013
8,1736.0,1309.0,1,0.993,0.109
9,1734.0,1243.0,1,0.997,0.052


## Key Takeaways

1. **Web scraping** uses `requests` to fetch HTML pages and `BeautifulSoup` to parse them
2. The Swiss TT website has a **search endpoint** to find players by name
3. Each player has a unique **ranking ID** in their URL
4. Match data is in **HTML tables** with consistent column structure
5. We use **regex** to validate dates and extract IDs from URLs
6. Adding **delays** (`time.sleep`) is important to not overload the server